In [1]:
# http://www.cse.yorku.ca/~amana/research/grid.pdf

In [6]:
import numpy as np
from numpy.linalg import norm
from numpy import array as arr
from numpy import floor, sign

import ipyvolume as ipv
from numpy import max, min, floor

Start = arr([49.66, 3.45, 18.56])
Slope = arr([0.09, -0.30, -0.39])

In [3]:
def box(o):
    p = arr([
        o.T,
        o.T + [0,0,1],
        o.T + [1,0,1],
        o.T + [1,0,0],
        o.T + [0,0,0],
        o.T + [0,1,0],
        o.T + [0,1,1],
        o.T + [0,0,1],
        o.T + [0,1,1],
        o.T + [1,1,1],
        o.T + [1,0,1],
        o.T + [1,1,1],
        o.T + [1,1,0],
        o.T + [1,0,0],
        o.T + [1,1,0],
        o.T + [0,1,0],
    ]).T
    ipv.plot(*p)

In [7]:
def sign0(x): return 1 if (x > 0) else 0

def transverse(U, V):
    V = V/norm(V)
    X, Y, Z = floor(U[0]), floor(U[1]), floor(U[2])
    dx, dy, dz = V[0], V[1], V[2]

    stepX = sign(V[0])
    stepY = sign(V[1])
    stepZ = sign(V[2])

    tDeltaX = 1/abs(dx) if (dx != 0) else 1e4
    tDeltaY = 1/abs(dy) if (dy != 0) else 1e4
    tDeltaZ = 1/abs(dz) if (dz != 0) else 1e4
    
    tMaxX = (sign0(dx) + sign(dx) * (X-U[0])) * tDeltaX if (dx != 0) else 1e4
    tMaxY = (sign0(dy) + sign(dy) * (Y-U[1])) * tDeltaY if (dy != 0) else 1e4
    tMaxZ = (sign0(dz) + sign(dz) * (Z-U[2])) * tDeltaZ if (dz != 0) else 1e4
    
    out = [arr([X,Y,Z])]
    for i in range(0, 3):
        if tMaxX < tMaxY:
            if tMaxX < tMaxZ:
                X += stepX
                tMaxX += tDeltaX
            else:
                Z += stepZ
                tMaxZ += tDeltaZ
        else:
            if tMaxY < tMaxZ:
                Y += stepY
                tMaxY += tDeltaY
            else:
                Z += stepZ
                tMaxZ += tDeltaZ
        out.append(arr([X,Y,Z]))   
    
    return out

ipv.clear()
fig = ipv.figure(width=950, height=600)

out = arr(transverse(Start, Slope))

[box(o) for o in out]
ipv.plot(*zip(Start, Start + Slope*15), color='blue')

ipv.xlim(min(out, axis=0)[0]-1, max(out, axis=0)[0]+1)
ipv.ylim(min(out, axis=0)[1]-1, max(out, axis=0)[1]+1)
ipv.zlim(min(out, axis=0)[2]-1, max(out, axis=0)[2]+1)
ipv.zlim(15.5, 18.5)
ipv.ylim(1.5, 3.5)
ipv.pylab.style.box_off()
ipv.show()

In [5]:
import ipyvolume.embed
ipyvolume.embed.embed_html("voxel raycast.html", fig, offline=True)